In [2]:
##Importing libraries
import nltk
import numpy as np
import pandas as pd
from collections import Counter
import re , nltk
from nltk import word_tokenize, sent_tokenize
import re
from nltk.corpus import stopwords, words, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker  # much faster

# Download NLTK resources (only first time)
nltk.download('punkt') #punctuations
nltk.download('stopwords')  ## list of stopwords
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\muhammad
[nltk_data]     waseem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\muhammad
[nltk_data]     waseem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to C:\Users\muhammad
[nltk_data]     waseem\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\muhammad
[nltk_data]     waseem\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df=pd.read_csv(r'C:\Users\muhammad waseem\OneDrive\Desktop\resume_analysis\resumes_clean_spellcheck_chunked.csv')

In [4]:
df.head()

,Resume,Category,clean_text
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,administrator marketing associate administrato...
1,"HR SPECIALIST, US HR OPERATIONS ...",HR,specialist operations summary versatile media ...
2,HR DIRECTOR Summary Over 2...,HR,director summary over years experience recruit...
3,HR SPECIALIST Summary Dedica...,HR,specialist summary dedicated driven and dynami...
4,HR MANAGER Skill Highlights ...,HR,manager skill highlights skills department sta...


In [5]:
df.isna().sum()

Resume        0
Category      0
clean_text    9
dtype: int64

In [6]:
df=df.dropna()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2526 entries, 0 to 2534
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Resume      2526 non-null   object
 1   Category    2526 non-null   object
 2   clean_text  2526 non-null   object
dtypes: object(3)
memory usage: 78.9+ KB


In [8]:
df['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      119
BUSINESS-DEVELOPMENT                                                                                                                                                                                                                                                                                                                           

In [9]:

# Print the number of rows before cleaning
print(f"Original number of rows: {len(df)}")

# Remove any rows where the 'Category' contains an '@' symbol
df_clean = df[~df['Category'].str.contains('@', na=False)].copy()

# Optional but recommended: Remove categories that appear less than 10 times
# This gets rid of the long tail of noisy, rare, or mistaken labels.
counts = df_clean['Category'].value_counts()
df_clean = df_clean[df_clean['Category'].isin(counts[counts >= 10].index)].copy()

print(f"Number of rows after filtering: {len(df_clean)}")

Original number of rows: 2526
Number of rows after filtering: 2457


In [11]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2457 entries, 0 to 2534
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Resume      2457 non-null   object
 1   Category    2457 non-null   object
 2   clean_text  2457 non-null   object
dtypes: object(3)
memory usage: 76.8+ KB


In [12]:
from sklearn.preprocessing import LabelEncoder
lc=LabelEncoder()
y=lc.fit_transform(df_clean['Category'])
print(list(zip(df_clean['Category'],y)))

[('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np.int64(19)), ('HR', np

In [17]:
from sklearn.model_selection import train_test_split
# from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB,ComplementNB,BernoulliNB
X_train,X_test,y_train,y_test=train_test_split(df_clean['clean_text'],y,test_size=0.2,random_state=42)
tf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features=10000)
X_train=tf.fit_transform(X_train)
X_test=tf.transform(X_test)
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
model=MultinomialNB(alpha=0.1)
model.fit(X_train_resampled,y_train_resampled)
prd=model.predict(X_test)
# print('prediction: ',lc.inverse_transform(prd))

In [18]:
from sklearn.metrics import accuracy_score,classification_report
accury=accuracy_score(y_test,prd)
accury

0.5833333333333334

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
import numpy as np

# Assuming X_train_resampled and y_train_resampled are already defined from your code:
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## 1. Define the model and the parameter grid
# We are testing different 'alpha' values for the smoothing parameter
model_mnb = MultinomialNB()
param_grid = {
    'alpha': np.logspace(-3, 0, 100) # Creates 100 values between 0.001 and 1.0
}

## 2. Set up GridSearchCV
# We use 'f1_weighted' as the scoring metric because accuracy is often misleading
# in imbalanced or multi-class scenarios, even after SMOTE.
grid_search = GridSearchCV(
    estimator=model_mnb,
    param_grid=param_grid,
    scoring='f1_weighted', # Use F1-score for better evaluation
    cv=5, # 5-fold cross-validation
    n_jobs=-1, # Use all available cores
    verbose=1
)

## 3. Fit and find the best parameters
print("Starting Grid Search...")
grid_search.fit(X_train_resampled, y_train_resampled)
print("Grid Search complete.")

## 4. Get the best results
best_alpha = grid_search.best_params_['alpha']
print(f"**Best Alpha (smoothing parameter): {best_alpha:.4f}**")
print(f"Best cross-validation F1-weighted score: {grid_search.best_score_:.4f}")

## 5. Use the best model for prediction
best_model = grid_search.best_estimator_
prd_optimized = best_model.predict(X_test)

# You will need to re-import/re-define lc (LabelEncoder or similar)
# print('Prediction with Optimized MNB: ', lc.inverse_transform(prd_optimized))

# You should also calculate the accuracy and other metrics here
from sklearn.metrics import accuracy_score, classification_report
print(f"\nAccuracy Score on Test Set: {accuracy_score(y_test, prd_optimized):.4f}")
print("\nClassification Report on Test Set:")
print(classification_report(y_test, prd_optimized))

Starting Grid Search...
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Grid Search complete.
**Best Alpha (smoothing parameter): 0.0014**
Best cross-validation F1-weighted score: 0.6744

Accuracy Score on Test Set: 0.5467

Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.81      0.81      0.81        31
           1       0.36      0.40      0.38        30
           2       0.80      0.40      0.53        10
           3       0.43      0.18      0.25        17
           4       0.44      0.18      0.26        22
           5       0.00      0.00      0.00         6
           6       0.53      0.45      0.49        22
           7       0.62      0.62      0.62        21
           8       0.00      0.00      0.00         6
           9       0.44      0.52      0.47        27
          10       0.90      0.75      0.82        24
          11       0.81      0.85      0.83        20
          12       0.13

c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Define the model and the parameter grid
# 'C' is the inverse of regularization strength. Smaller C means MORE regularization.
# We test a wide range to find the optimal balance between bias and variance.
model_lr = LogisticRegression(
    solver='liblinear', # Good for small datasets and L1/L2 penalties
    penalty='l2',        # Start with L2 regularization
    max_iter=1000,
    random_state=42
)
param_grid_lr = {
    # Test C values from very regularized (0.01) to less regularized (10.0)
    'C': np.logspace(-2, 1, 10) 
}

## Set up GridSearchCV
print("Starting Logistic Regression Grid Search...")
grid_search_lr = GridSearchCV(
    estimator=model_lr,
    param_grid=param_grid_lr,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1,
    verbose=1
)

## Fit and find the best parameters (using your resampled data)
# X_train_resampled, y_train_resampled are used for fitting
grid_search_lr.fit(X_train_resampled, y_train_resampled)
print("Grid Search complete.")

## Get the best results
best_C = grid_search_lr.best_params_['C']
best_lr_score = grid_search_lr.best_score_
print(f"\n**Best C (Regularization Strength): {best_C:.4f}**")
print(f"Best cross-validation F1-weighted score: {best_lr_score:.4f}")

## Evaluate on the Test Set
best_model_lr = grid_search_lr.best_estimator_
prd_lr = best_model_lr.predict(X_test)

print(f"\nAccuracy Score on Test Set (Logistic Regression): {accuracy_score(y_test, prd_lr):.4f}")
print("Classification Report on Test Set:")
print(classification_report(y_test, prd_lr))

Starting Logistic Regression Grid Search...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(


Grid Search complete.

**Best C (Regularization Strength): 10.0000**
Best cross-validation F1-weighted score: 0.7703

Accuracy Score on Test Set (Logistic Regression): 0.7256
Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.85      0.90      0.88        31
           1       0.78      0.60      0.68        30
           2       0.45      0.50      0.48        10
           3       0.56      0.29      0.38        17
           4       0.78      0.32      0.45        22
           5       0.00      0.00      0.00         6
           6       0.74      0.64      0.68        22
           7       0.74      0.81      0.77        21
           8       1.00      0.33      0.50         6
           9       0.86      0.89      0.87        27
          10       0.86      0.75      0.80        24
          11       0.83      0.95      0.88        20
          12       0.68      0.62      0.65        24
          13       0.86      0.86

In [15]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Define the model and the parameter grid
# C is the inverse of regularization strength, similar to Logistic Regression
model_svc = LinearSVC(
    dual=False, # Set to False when n_samples > n_features, which is often true after TF-IDF
    max_iter=10000,
    random_state=42
)
param_grid_svc = {
    # Test C values from very regularized (0.01) to less regularized (10.0)
    'C': np.logspace(-2, 1, 10)
}

## Set up GridSearchCV
print("\nStarting LinearSVC Grid Search...")
grid_search_svc = GridSearchCV(
    estimator=model_svc,
    param_grid=param_grid_svc,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1,
    verbose=1
)

## Fit and find the best parameters (using your resampled data)
# X_train_resampled, y_train_resampled are used for fitting
grid_search_svc.fit(X_train_resampled, y_train_resampled)
print("Grid Search complete.")

## Get the best results
best_C_svc = grid_search_svc.best_params_['C']
best_svc_score = grid_search_svc.best_score_
print(f"\n**Best C (Regularization Strength): {best_C_svc:.4f}**")
print(f"Best cross-validation F1-weighted score: {best_svc_score:.4f}")

## Evaluate on the Test Set
best_model_svc = grid_search_svc.best_estimator_
prd_svc = best_model_svc.predict(X_test)

print(f"\nAccuracy Score on Test Set (LinearSVC): {accuracy_score(y_test, prd_svc):.4f}")
print("Classification Report on Test Set:")
print(classification_report(y_test, prd_svc))


Starting LinearSVC Grid Search...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Grid Search complete.

**Best C (Regularization Strength): 0.0215**
Best cross-validation F1-weighted score: 0.7562

Accuracy Score on Test Set (LinearSVC): 0.6931
Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.93      0.90      0.92        31
           1       0.69      0.67      0.68        30
           2       0.38      0.60      0.46        10
           3       0.36      0.24      0.29        17
           4       0.56      0.45      0.50        22
           5       0.00      0.00      0.00         6
           6       0.78      0.64      0.70        22
           7       0.64      0.67      0.65        21
           8       0.00      0.00      0.00         6
           9       0.86      0.70      0.78        27
          10       0.85      0.71      0.77        24
          11       0.78      0.70      0.74        20
  

c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape

In [43]:
from sklearn.decomposition import TruncatedSVD
from imblearn.over_sampling import SMOTE

# 1. Initialize Truncated SVD
# We aim to reduce the thousands of features to a manageable, dense set.
# Starting with 300 components is a common practice for text data.
n_components = 1500
svd = TruncatedSVD(n_components=n_components, random_state=42)

# 2. Fit and Transform the Training Data
# X_train is your original sparse TF-IDF matrix (before SMOTE)
X_train_svd = svd.fit_transform(X_train) 

# 3. Transform the Test Data
X_test_svd = svd.transform(X_test)

# Check the variance explained by the components (optional but useful)
print(f"Variance explained by {n_components} components: {svd.explained_variance_ratio_.sum():.4f}")

# 4. Apply SMOTE to the new, dense feature set
smote = SMOTE(random_state=42)
X_train_resampled_dense, y_train_resampled = smote.fit_resample(X_train_svd, y_train)

print(f"Original features: {X_train.shape[1]}")
print(f"Reduced features: {X_train_svd.shape[1]}")
print(f"SMOTE Resampled features: {X_train_resampled_dense.shape}")

Variance explained by 1500 components: 0.9823
Original features: 10000
Reduced features: 1500
SMOTE Resampled features: (2304, 1500)


In [ ]:
# import numpy as np
# import gensim.downloader as api
# from sklearn.feature_extraction.text import TfidfVectorizer
# from imblearn.over_sampling import SMOTE
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, classification_report
# import re
# import scipy.sparse

# # --- I. Setup ---
# # 1. Download and load the Gensim model (same as before)
# print("Downloading and loading Gensim model...")
# glove_model = api.load("glove-twitter-25")
# embedding_dim = 25

# # 2. Fit the TfidfVectorizer (to get word weights)
# # X_train is your original clean text data
# tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
# tfidf.fit(X_train)
# # Get the vocabulary and its corresponding index mapping from TFIDF
# word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))


# # --- II. Weighted Vectorization Function ---
# def weighted_document_vector(text, model, tfidf_index, embedding_dim):
#     # Basic cleaning and tokenization
#     tokens = re.sub(r'[^a-zA-Z\s]', '', str(text).lower()).split()
    
#     # Store the weighted sum and total weight
#     weighted_sum = np.zeros(embedding_dim)
#     total_weight = 0

#     for token in tokens:
#         # Check if the word is in BOTH GloVe (for vector) and TFIDF (for weight)
#         if token in model and token in tfidf_index:
#             weight = tfidf_index[token]
#             vector = model[token]
            
#             weighted_sum += vector * weight
#             total_weight += weight

#     if total_weight == 0:
#         return np.zeros(embedding_dim)
        
#     return weighted_sum / total_weight

# # --- III. Execution ---

# # 1. Create Weighted Embedding Feature Sets
# X_train_weighted_emb = np.array([weighted_document_vector(text, glove_model, word2tfidf, embedding_dim) for text in X_train])
# X_test_weighted_emb = np.array([weighted_document_vector(text, glove_model, word2tfidf, embedding_dim) for text in X_test])

# print(f"Weighted Embedding features created. Train Shape: {X_train_weighted_emb.shape}")

# # 2. Apply SMOTE to the dense weighted features
# smote = SMOTE(random_state=42)
# X_train_resampled_weighted, y_train_resampled = smote.fit_resample(X_train_weighted_emb, y_train)

# # 3. Train LinearSVC (Your champion model)
# model_svc_weighted = LinearSVC(C=1.0, dual=True, max_iter=10000, random_state=42)
# print("\nStarting LinearSVC training on Weighted Embeddings...")
# model_svc_weighted.fit(X_train_resampled_weighted, y_train_resampled)

# # 4. Evaluate
# prd_svc_weighted = model_svc_weighted.predict(X_test_weighted_emb)

# print("\n--- LinearSVC with TF-IDF Weighted Embeddings Results ---")
# print(f"Accuracy Score: {accuracy_score(y_test, prd_svc_weighted):.4f}")
# print("Classification Report:")
# print(classification_report(y_test, prd_svc_weighted))

Weighted Embedding features created. Train Shape: (1965, 25)

Starting LinearSVC training on Weighted Embeddings...

--- LinearSVC with TF-IDF Weighted Embeddings Results ---
Accuracy Score: 0.4045
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.74      0.64        31
           1       0.62      0.17      0.26        30
           2       0.33      0.40      0.36        10
           3       0.11      0.12      0.11        17
           4       0.00      0.00      0.00        22
           5       0.12      0.17      0.14         6
           6       0.42      0.45      0.43        22
           7       0.21      0.14      0.17        21
           8       0.00      0.00      0.00         6
           9       0.48      0.41      0.44        27
          10       0.68      0.71      0.69        24
          11       0.48      0.60      0.53        20
          12       0.00      0.00      0.00        24
          13       0.6

c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\ProgramData\miniconda3\envs\data\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape

In [19]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.metrics import accuracy_score, classification_report

# 1. Define the model
model_svc_final = LinearSVC(dual=False, max_iter=10000, random_state=42)

# 2. Define the search space
# C: We tune C over a wide logarithmic range, focused around your best result (4.6416)
# penalty: We test both L1 (which forces some coefficients to zero, good for feature selection) 
#          and L2 (your current penalty).
param_distributions = {
    'C': loguniform(1e-2, 1e2), # Tune C from 0.01 to 100
    'penalty': ['l1', 'l2']
}

# 3. Set up Randomized Search CV
# n_iter=50 means we try 50 different random combinations.
# IMPORTANT: You must use X_train_resampled and y_train_resampled from your original TF-IDF + SMOTE step.
random_search = RandomizedSearchCV(
    estimator=model_svc_final,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

print("Starting Randomized Search on LinearSVC (TF-IDF features)...")
random_search.fit(X_train_resampled, y_train_resampled)
print("Search complete.")

# 4. Evaluate the best model
best_model_final = random_search.best_estimator_
prd_final = best_model_final.predict(X_test)

print(f"\n**Best C: {random_search.best_params_['C']:.4f}, Best Penalty: {random_search.best_params_['penalty']}**")
print(f"Accuracy Score (Tuned LinearSVC): {accuracy_score(y_test, prd_final):.4f}")
print("Classification Report:")
print(classification_report(y_test, prd_final))

Starting Randomized Search on LinearSVC (TF-IDF features)...
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Search complete.

**Best C: 2.4810, Best Penalty: l1**
Accuracy Score (Tuned LinearSVC): 0.7907
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94        31
           1       0.85      0.73      0.79        30
           2       0.67      0.60      0.63        10
           3       0.64      0.41      0.50        17
           4       0.71      0.55      0.62        22
           5       0.22      0.33      0.27         6
           6       0.89      0.73      0.80        22
           7       0.62      0.62      0.62        21
           8       1.00      0.67      0.80         6
           9       0.92      0.89      0.91        27
          10       1.00      0.88      0.93        24
          11       0.86      0.95      0.90        20
          12       0.75      0.88      0.81        24
 

In [22]:
import pickle as pkl
with open('resume_analy.pkl', 'wb') as f:
    pkl.dump(best_model_final, f)

In [21]:
import pickle

pickle.dump(tf, open("tfidf_vectorizer.pkl", "wb"))
